# Warning

This notebook uses instaloader, a powerful but dangerous instagram scraping library.  
Only use it with secondary accounts! There is a high risk of temporary or permanent bans.  
It is very useful for some initial or small data batches (maybe a post or so), but after that it gets dangerous.

A user on the instaloader GitHub had a good description here:
 > MHO this is a very important topic. Sharing my observation from 18 months or so using Instaloader's Python module.  
I noticed that to avoid triggering Instagram's bad behavior detectors, besides Instaloader's code behaving as similarly as possible to an Instagram client
(thank you repo maintainers), we users must be aware of, and avoid, behaviors that can trigger accounts and/or IP address blocks.

> Bad behavior detectors would be triggered even if it was Instagram's own app/webapp. For example, if you try to access some IG post URLs with anonymous sessions from a browser multiple times in a row, Instagram will route you to the login page. Keep doing it and it'll temporarily block your IP address. Keep insisting even more, and it may flag your IP as authenticated session only, no anonymous access. Keep doing it over and over, even with an authenticated session it'll flag your account, temporarily or for good. It's happened to my home IP address and many test accounts even before trying Instaloader.

> So, I've been able to reasonably to use Instaloader at scale by:
a) Authenticating accounts via Firefox with some automation via playright to collect the cookies;
b) Using a rotating residential proxy that allow multiple requests per session to scatter Instaloader sessions over multiple IP addresses and geographies (single request proxy connections won't work);
c) Limiting the rate of requests per account in authenticated sessions. Yes, it requires some effort to produce and take care of more and mores accounts if you need to frequently collect lots of stories, but it is what it is.

[Source](https://github.com/instaloader/instaloader/issues/1984#issuecomment-1696375143)

Also, to get instaloader to authenticate correctly, you need to modify the library files itself (at least in version 4.14).

Please go to the instaloadercontext.py file and remove `'sessionid': '',` from the `session.cookies.update()`method call dictionary in the `login()` method.
You also need to either use direct login via username and password or use a cookie from the browser directly to log in. Sessions are not working correctly as of now.

[Source](https://github.com/instaloader/instaloader/issues/2487#issue-2807621924)

In [ ]:
from instaloader import Instaloader, Profile
from my_secrets import INSTA_ACCOUNT_USERNAME, INSTA_ACCOUNT_PASSWORD 
L = Instaloader(user_agent="chrome")
L.login(INSTA_ACCOUNT_USERNAME, INSTA_ACCOUNT_PASSWORD)

In [ ]:
profile = Profile.from_username(L.context, "dorfterror")
profile.username, profile.full_name, profile.biography, profile.external_url, profile.followers, profile.followees, profile.is_verified

# Attention

The following code was just for experimenation and data exploration.  
Running the whole post scraping for loop on a profile will very likely get your banned.

### BE CAREFUL

In [ ]:
import sqlite3

conn = sqlite3.connect('instagram_posts.db')
c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        shortcode TEXT PRIMARY KEY,
        caption TEXT,
        likes INTEGER,
        comments_count INTEGER,
        comments TEXT,
        commenters TEXT,
        co_authors TEXT
    )
''')
conn.commit()



for i, post in enumerate(profile.get_posts()):
    print(f"Processing post: {post.shortcode}")
    shortcode = post.shortcode
    caption = post.caption
    likes = post.likes
    comments_count = post.comments
    print(f"Likes: {likes}, Comments: {comments_count}")

    comments_list = []
    commenters_list = []

    for comment in post.get_comments():
        comments_list.append(comment.text)
        commenters_list.append(comment.owner.username)

    c.execute('''
        INSERT OR REPLACE INTO posts (shortcode, caption, likes, comments_count, comments, commenters)
        VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        shortcode,
        caption,
        likes,
        comments_count,
        "\n".join(comments_list),
        ", ".join(commenters_list),
    ))

    conn.commit()
    if i == 5:  # Limit to first 5 posts for testing
        break

conn.close()


Processing post: DBavDJ7MpfB
Likes: 516, Comments: 14
Processing post: DCWBEZRM1TC
Likes: 221, Comments: 26
Processing post: DLxrfqfMSvp
Likes: 158, Comments: 2
Processing post: DLuXtgJs9W0
Likes: 114, Comments: 3
Processing post: DLj1jdAMTnt
Likes: 48, Comments: 0
Processing post: DLaONVUIGwb
Likes: 55, Comments: 3


In [15]:
print(post._owner_profile)

<Profile riez_openair (7160047809)>


In [12]:
temp = post._iphone_struct
print(temp)

{'code': 'DLaONVUIGwb', 'pk': '3664303735709920283', 'id': '3664303735709920283_7160047809', 'ad_id': None, 'boosted_status': None, 'boost_unavailable_identifier': None, 'boost_unavailable_reason': None, 'caption': {'has_translation': True, 'created_at': 1751040009, 'pk': '18090292417629160', 'text': 'Gute vier Wochen sind es noch bis zu eurer Lieblingsgartenparty in der Jubiläumsedition und am Freitag werdet ihr dort auf @dorfterror treffen. Hört euch an was Fussel zu sagen hat und checkt sie auf den gängigen Kanälen aus falls ihr das noch nicht getan habt, es lohnt sich. Riez Open Air 2025: 24. - 26.07.2025. Ticketshop, Hörprobe, Infos findet ihr unter https://linktr.ee/riezopenair #roa2025 #riezopenair festivalgartenparty punkrock hardcore #itchy #montreal #jupiterjones #bane #crystallake #captainplanet #themeffs #timvantol #dunerats #knochenfabrik #desaster #bubonix #thepill #waronwomen #theironroses #phantombay #indecentbehavior #christmas #theredflags #statepower #fuckingangry #d

In [ ]:
from pprint import pprint
pprint(post._asdict())

{'__isXDTGraphMediaInterface': 'XDTGraphVideo',
 '__typename': 'GraphVideo',
 'accessibility_caption': None,
 'can_see_insights_as_brand': False,
 'caption': 'Gute vier Wochen sind es noch bis zu eurer Lieblingsgartenparty '
            'in der Jubiläumsedition und am Freitag werdet ihr dort auf '
            '@dorfterror treffen. Hört euch an was Fussel zu sagen hat und '
            'checkt sie auf den gängigen Kanälen aus falls ihr das noch nicht '
            'getan habt, es lohnt sich. Riez Open Air 2025: 24. - 26.07.2025. '
            'Ticketshop, Hörprobe, Infos findet ihr unter '
            'https://linktr.ee/riezopenair #roa2025 #riezopenair '
            'festivalgartenparty punkrock hardcore #itchy #montreal '
            '#jupiterjones #bane #crystallake #captainplanet #themeffs '
            '#timvantol #dunerats #knochenfabrik #desaster #bubonix #thepill '
            '#waronwomen #theironroses #phantombay #indecentbehavior '
            '#christmas #theredflags #statep

In [ ]:
followers = profile.get_followers()

data = []
for follower in followers:
    data.append({
        "id": follower.userid,
        "username": follower.username,
        "full_name": follower.full_name,
        "follower_count": follower.followers,
        "post_count": follower.mediacount
    })
    if len(data) >= 10:  # Limit to first 10 followers for testing
        break
print(data)

In [ ]:
from pprint import pprint
pprint(follower._asdict())

{'ai_agent_type': None,
 'bio_links': [],
 'biography': 'life enthusiast and fellow musician\n'
              'singing in @bulletinsideband band',
 'biography_with_entities': {'entities': [{'hashtag': None,
                                           'user': {'username': 'bulletinsideband'}}],
                             'raw_text': 'life enthusiast and fellow musician\n'
                                         'singing in @bulletinsideband band'},
 'blocked_by_viewer': False,
 'business_address_json': None,
 'business_category_name': None,
 'business_contact_method': 'UNKNOWN',
 'business_email': None,
 'business_phone_number': None,
 'category_enum': None,
 'category_name': 'Musician/band',
 'country_block': False,
 'edge_follow': {'count': 1292},
 'edge_followed_by': {'count': 359},
 'edge_mutual_followed_by': {'count': 0, 'edges': []},
 'eimu_id': '17843885537184327',
 'external_url': None,
 'external_url_linkshimmed': None,
 'fb_profile_biolink': None,
 'fbid': '17841435529801973

In [ ]:
comments = post.get_comments()
print(comments[0].answers)